## 0721 딥러닝 정규화 및 최적화 실습  Solution (02-01 ~ 02-03)


#### 02-01 Optimizer Solution

1)
```python
#It could be changeable. An integer number between 1 and 60,000 (train data size)
batch_size = 1024 
```

2)
```python
optimizer = tf.train.AdamOptimizer(learning_rate) #Or any optimizer

```
3)
```python
optimizers={'sgd': tf.train.GradientDescentOptimizer(learning_rate), 'sgd w/ nesterov': tf.train.MomentumOptimizer(learning_rate, momentum=0.3 ,use_nesterov=True), 'adagrad': tf.train.AdagradOptimizer(learning_rate), 'rmsprop': tf.train.RMSPropOptimizer(learning_rate), 'adam': tf.train.AdamOptimizer(learning_rate)}

train_loss_histories = []
train_acc_histories = []
test_acc_histories = []
for opt_name in optimizers:
    optimizer = optimizers[opt_name]
    train_loss_history = []
    train_acc_history = []
    test_acc_history = []
    start_time = time()
    for epoch in range(epochs):
        for (idx, (images, labels)) in enumerate(train_dataset.take(iterations)):
           
            # gradient를 계산하고 optimizer를 이용해 back propagation을 수행합니다.
            grads = grad(model, images, labels)
            
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
       
            # performance를 계산합니다
            train_loss = loss_fn(model, images, labels)
            train_accuracy = accuracy_fn(model, images, labels)
            test_accuracy = accuracy_fn(model, x_test, y_test)
            # 그래프를 그리기 위해 기록합니다.
            train_loss_history.append(train_loss.numpy())
            train_acc_history.append(train_accuracy.numpy())
            test_acc_history.append(test_accuracy.numpy())

            # 학습 과정을 출력합니다.
            if idx % 20 == 0 or idx == iterations - 1:
                print("Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch + 1, idx + 1, iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
    train_loss_histories.append(train_loss_history)
    train_acc_histories.append(train_acc_history)
    test_acc_histories.append(test_acc_history)
plot_fn([key for key in optimizers], train_loss_histories, train_acc_histories, test_acc_histories)

```


#### 02-01 Learning Rate

1) Learning Rate를 10^-1, 10^-2 , ..., 10^-6으로 list를 만들어 실행하십시오

```python
learning_rates = [10**(-i) for i in range(1,6)]
```

2) Sceduler Function를 구현하십시오

```python
# lr: current learning rate
#@TODO: learning rate decay 구현
##30 epcoh 마다 Learning Rate가 2씩 나눠지도록 구현하십시오
def schedule(epoch, lr):
    lr: current learning rate
    new_lr = lr
    if epoch > 0 and epoch % 30 == 0:
        new_lr = lr * 0.5
    return new_lr
```

#### 02-03 optimization-Grid Search

1) # 가장 좋은 validation accuracy를 보인 하이퍼파라미터로 model을 create 하고 training data를 다시 학습합니다. 위에서 보인 가장 좋은 Hyperparameter를 직접 입력해주어도 되고, grid_result.best_params_ 이라는 함수를 사용해도 됩니다.


```python
best_model = create_model(init=grid_result.best_params_['init'], optimizer=grid_result.best_params_['optimizer'])

best_hist = best_model.fit(x_train, y_train, batch_size=grid_result.best_params_['batch_size'], epochs=grid_result.best_params_['nb_epoch'], validation_data=(x_test, y_test))

```